In [3]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import torch                    # Pytorch module 
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders 
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
from torchvision.utils import make_grid       # for data checking
from torchvision import datasets, transforms, models
# for working with classes and images
from torchsummary import summary              # for getting the summary of our model

%matplotlib inline

In [4]:
data_dir = "./PlantVillage-Dataset-processed"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"
sunny_dir = data_dir + "/sunny"
cloudy_dir = data_dir + "/cloudy"
gan_dir_sunny = data_dir + "/generated_dataset2"


diseases = os.listdir(train_dir)

In [10]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.GaussianBlur(kernel_size=(7, 13), sigma=(9, 11)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_gan = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
sunny_dataset = datasets.ImageFolder(root=sunny_dir, transform=transform)
cloudy_dataset = datasets.ImageFolder(root=cloudy_dir, transform=transform)
gan_dataset_sunny = datasets.ImageFolder(root=gan_dir_sunny, transform=transform_gan)


batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
cloudy_loader = torch.utils.data.DataLoader(cloudy_dataset, batch_size=batch_size, shuffle=False)
sunny_loader = torch.utils.data.DataLoader(sunny_dataset, batch_size=batch_size, shuffle=False)
gan_sunny_loader = torch.utils.data.DataLoader(gan_dataset_sunny, batch_size=batch_size, shuffle=False)



In [4]:
### DONOT RUN THIS CELL
class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResNet9, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.res1 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1))

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.res2 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                  nn.ReLU(),
                                  nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        # Residual block
        r = self.res1(x)
        x = F.relu(x + r)
        
        x = self.pool(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x))
        # Residual block
        r = self.res2(x)
        x = F.relu(x + r)

        x = self.classifier(x)
        return x


In [33]:
### DONOT RUN THIS CELL
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet9(in_channels=3, num_classes=len(train_dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
        
        # Evaluate
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy: {100 * correct / total}%')

torch.cuda.empty_cache()
train(model, criterion, optimizer, train_loader, test_loader, epochs=10)



In [6]:
from torchvision.models import resnet18, ResNet18_Weights

# Load a pretrained ResNet model
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all layers in the model to prevent them from being updated during training
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer with a new one that matches the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(train_dataset.classes))


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)  # Only optimize the final layer

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/1, Loss: 0.6924


In [8]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')


Accuracy on test set: 91.24678190511217%


In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in sunny_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on sunny set: {100 * correct / total}%')

Accuracy on sunny set: 62.76204486943729%


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in gan_sunny_loader:
        # print(inputs,labels)
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on sunny after GAN set: {100 * correct / total}%')

Accuracy on sunny after GAN set: 17.63157894736842%


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in cloudy_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on cloudy set: {100 * correct / total}%')

Accuracy on cloudy set: 42.84663479220301%
